In [ ]:
# %% Calculus 2 - Section 12.84
#    Code challenge: crating pdfs

# This code pertains to a calculus course provided by Mike X. Cohen on Udemy:
#   > https://www.udemy.com/course/pycalc2_x
# The code in this repository is developed to solve the exercises provided along
# the course, and it has been written partially indepentently and partially
# from the code developed by the course instructor.


In [1]:
import numpy                as np
import sympy                as sym
import matplotlib.pyplot    as plt
import matplotlib.colors    as mcolors
import scipy.integrate      as spi
import math
import mpmath
import plotly.graph_objects as go
import sympy.stats

from scipy.signal                     import find_peaks
from scipy                            import stats
from IPython.display                  import display,Math
from google.colab                     import files
from IPython.display                  import Audio
from scipy.io                         import wavfile
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')

import matplotlib.animation as animation
from matplotlib import rc
rc('animation', html='jshtml')


In [25]:
# %% Exercise 1
#    Compute the logistic pdf shown in the video using numpy; use a domain of
#    [-5,20] and see what happens with the integral

# Params
x     = np.linspace(-5,20,2001)
dx    = x[1]-x[0]
mu    = 0
sigma = 2

# Pdf
expo = np.exp(-(x-mu)/sigma)
pdf  = expo / (sigma*(1 + expo)**2)
pdf *= dx

# Confirm normalisation condition
sumPdf = np.sum(pdf)
print('sumPdf = %.2f' %sumPdf)

# Plot
phi = (1 + np.sqrt(5)) / 2
plt.figure(figsize=(phi*5,5))

plt.plot(x,pdf)
plt.axhline(y=0,color='grey',linestyle=':',linewidth=0.8)

plt.gca().set(xlabel='x',ylabel='Probability',title='Logistic (sum = %.4f)' %sumPdf)
plt.tight_layout()

plt.savefig('fig4_codechallenge_84_exercise_1.png')
plt.show()
files.download('fig4_codechallenge_84_exercise_1.png')


In [ ]:
# %% Exercise 2
#    Compare numpy logistic to scipy's

logistic_pdf_sp  = stats.logistic.pdf(x,loc=mu,scale=sigma)
logistic_pdf_sp *= dx

# Plot
phi = (1 + np.sqrt(5)) / 2
plt.figure(figsize=(phi*5,5))

plt.plot(x[::25], pdf[::25],'o',label='numpy')
plt.plot(x,logistic_pdf_sp,label='scipy')
plt.axhline(y=0,color='grey',linestyle=':',linewidth=0.8)

plt.gca().set(xlabel='x',ylabel='Probability',xlim=[x[0]-.5,x[-1]+.5],title=f'Logistic (sum = {np.sum(logistic_pdf_sp):.4f})')
plt.legend()

plt.savefig('fig4_codechallenge_84_exercise_2.png')
plt.show()
files.download('fig4_codechallenge_84_exercise_2.png')


In [ ]:
# %% Exercise 3
#    Compute the analytical integral beween 0 and pi

# Sympy expression
xx = sym.symbols('x')
L  = sym.stats.Logistic('L',sym.sympify(mu),sym.sympify(sigma))
logistic_pdf_sym = sym.stats.density(L)(xx)

display(Math('p(x) = %s' %sym.latex(logistic_pdf_sym)))
print()

# Definite integral
a = 0
b = sym.pi

exact_int = sym.integrate(logistic_pdf_sym,(xx,a,b))

display(Math('%s \\;=\\; %s' %(sym.latex(sym.Integral(logistic_pdf_sym,(xx,a,b))),sym.latex(exact_int))))
print()

# Evaluate numerically
logistic_pdf_l = sym.lambdify(xx,logistic_pdf_sym)
pdf_sym = logistic_pdf_l(x) * dx

# Plot
phi = (1 + np.sqrt(5)) / 2
plt.figure(figsize=(phi*5,5))

plt.plot(x,pdf_sym,'tab:red',label='Logistic pdf')
x4defint = (x>=a) & (x<=b)
plt.fill_between(x[x4defint],pdf_sym[x4defint],color='tab:red',edgecolor='none',alpha=.5,label=f'Area = {float(exact_int):.3f}')
plt.axhline(y=0,color='grey',linestyle=':',linewidth=0.8)

plt.title('Logistic pdf and its definite integral')
plt.legend()
plt.gca().set(xlabel='x',ylabel='pdf',xlim=[x[0]-.5,x[-1]+.5])

plt.savefig('fig5_codechallenge_84_exercise_3.png')
plt.show()
files.download('fig5_codechallenge_84_exercise_3.png')


In [ ]:
# %% Exercise 4
#    Compute the numerical integral beween 0 and pi (use Simpson)

# Specify dx correctly (scipy already scales by dx, so we have to undo the
# scaling we did in exercise 2)
approx_int = spi.simpson(logistic_pdf_sp[x4defint],dx=dx/dx)

print(f'Area from sympy = {float(exact_int):.8f}')
print(f'Area from scipy = {approx_int:.8f}')
